<a href="https://colab.research.google.com/github/rhiats/mimicIII/blob/main/Analysis_of_Patients_with_Type_2_Diabetes_in_ICU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Stratification Analysis of Patients with Type 2 Diabetes in ICU**

Person diagnosed with Type 2 Diabetes in the year 2008 and greater than equal to 65 years old.

Type 2 Diabetic ICD9 Codes (250)/ Admissions - 'diabet...'.

- Patients - distribution of age
- Admissions- breakdown of insurance type. (one hot encode)
- Admissions - Probability of having diabetes and being male/female (one hot encode)
- Prescriptions  - Top 3 drugs (one hot encode)
- Prescriptions - Distribtuion number of drugs per patient
- ICUSTAYS - Distribution of time in ICU
- Open (DIAGNOSES_ICD.csv.gz)

http://www.icd9data.com/2014/Volume1/240-279/249-259/250/250.00.htm

https://physionet.org/content/mimic3-carevue/1.4/

https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/

In [2]:
import pandas as pd

In [21]:
df=pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-carevue-subset-1.4/ADMISSIONS.csv.gz', compression='gzip')

def diabetetesFind(x):
  """
    Find patients diagnosed with Diabetes.
    @p: pandas series 'diagnosis'
    @r: dataframe with diabetic patients
  """
  x=str(x)
  x=x.lower()
  return x

df['diagnosis_lower']=df['diagnosis'].apply(diabetetesFind)
df.head()

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,diagnosis_lower
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN,NaN,NaN,NEWBORN,0,1,newborn
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,1,hypotension
2,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,BUDDHIST,NaN,ASIAN,NaN,NaN,NEWBORN,0,1,newborn
3,6,7,118037,2121-05-23 15:05:00,2121-05-27 11:57:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1,newborn
4,7,8,159514,2117-11-20 10:22:00,2117-11-24 14:20:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1,newborn


In [2]:
#ICD Codes to gather the patient id and get all their information for the year 2008
#Year
#Age
#Sex

**PCA**

**Scree Plot**

Select Feature 1

**Interpret the Weights of the First Component**

**Project each patient on the First Component to Calculate their Severity Score**

In the future, patients can be projected on the component to determine their risk for developing Diabetes based on pre-existing conditions.